In [1]:
# v 1.8.0 

In [2]:
import tensorflow as tf
import numpy as np

/Users/admin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
char_rdic = ['h', 'e', 'l', 'o'] # id -> char
char_dic = {w:i for i, w in enumerate(char_rdic)} # char -> id
print(char_dic)

{'h': 0, 'e': 1, 'l': 2, 'o': 3}


In [4]:
ground_truth = [char_dic[c] for c in 'hello']
print(ground_truth)

[0, 1, 2, 2, 3]


In [5]:
x_data = np.array([[1, 0, 0, 0],
                  [0, 1, 0, 0],
                  [0, 0, 1, 0],
                  [0, 0, 1, 0]],
                 dtype = 'f')

In [6]:
x_data = tf.one_hot(ground_truth[:-1], len(char_dic), 1.0, 0.0, -1)
print(x_data)

Tensor("one_hot:0", shape=(4, 4), dtype=float32)


In [7]:
# Configuration

In [8]:
rnn_size = len(char_dic)
batch_size = 1
output_size = 4

In [9]:
print(ground_truth[:-1])

[0, 1, 2, 2]


In [10]:
# RNN Model
rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units = rnn_size,
                                       #input_size = None, # deprecated at tensorflow 0.9
                                       #activation = tanh,
                                       )
print(rnn_cell)

In [11]:
initial_state = rnn_cell.zero_state(batch_size, tf.float32)
print(initial_state)

Tensor("BasicRNNCellZeroState/zeros:0", shape=(1, 4), dtype=float32)


In [12]:
initial_state_1 = tf.zeros([batch_size, rnn_cell.state_size])
print(initial_state_1)

Tensor("zeros:0", shape=(1, 4), dtype=float32)


In [13]:
x_split = tf.split(x_data, len(char_dic), 0)
print(x_split)

[<tf.Tensor 'split:0' shape=(1, 4) dtype=float32>, <tf.Tensor 'split:1' shape=(1, 4) dtype=float32>, <tf.Tensor 'split:2' shape=(1, 4) dtype=float32>, <tf.Tensor 'split:3' shape=(1, 4) dtype=float32>]


In [14]:
tf.__version__

'1.8.0'

In [15]:
outputs, state = tf.contrib.rnn.static_rnn(cell = rnn_cell, inputs = x_split, initial_state = initial_state)

In [16]:
print(outputs)
print(state)

[<tf.Tensor 'rnn/basic_rnn_cell/Tanh:0' shape=(1, 4) dtype=float32>, <tf.Tensor 'rnn/basic_rnn_cell/Tanh_1:0' shape=(1, 4) dtype=float32>, <tf.Tensor 'rnn/basic_rnn_cell/Tanh_2:0' shape=(1, 4) dtype=float32>, <tf.Tensor 'rnn/basic_rnn_cell/Tanh_3:0' shape=(1, 4) dtype=float32>]
Tensor("rnn/basic_rnn_cell/Tanh_3:0", shape=(1, 4), dtype=float32)


In [17]:
logits = tf.reshape(tf.concat(outputs, 1),
     [-1, rnn_size])
logits.get_shape()

TensorShape([Dimension(4), Dimension(4)])

In [18]:
targets = tf.reshape(ground_truth[1:], [-1])
targets.get_shape()

TensorShape([Dimension(4)])

In [19]:
weights = tf.ones([len(char_dic) * batch_size])

In [20]:
print(weights)

Tensor("ones:0", shape=(4,), dtype=float32)


In [21]:
loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], [targets], [weights])
cost = tf.reduce_sum(loss) / batch_size
train_op = tf.train.RMSPropOptimizer(0.01, 0.9).minimize(cost)

In [22]:
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    for i in range(100):
        sess.run(train_op)
        result = sess.run(tf.argmax(logits, 1))
        print(result, [char_rdic[t] for t in result])
print('end')

Instructions for updating:
Use `tf.global_variables_initializer` instead.
[2 1 0 2] ['l', 'e', 'h', 'l']
[2 3 0 2] ['l', 'o', 'h', 'l']
[2 3 0 2] ['l', 'o', 'h', 'l']
[2 3 0 2] ['l', 'o', 'h', 'l']
[2 3 0 2] ['l', 'o', 'h', 'l']
[2 3 0 2] ['l', 'o', 'h', 'l']
[2 3 0 2] ['l', 'o', 'h', 'l']
[2 3 3 2] ['l', 'o', 'o', 'l']
[2 3 3 2] ['l', 'o', 'o', 'l']
[2 3 3 2] ['l', 'o', 'o', 'l']
[2 3 3 2] ['l', 'o', 'o', 'l']
[2 3 3 2] ['l', 'o', 'o', 'l']
[2 3 3 2] ['l', 'o', 'o', 'l']
[2 3 3 3] ['l', 'o', 'o', 'o']
[2 3 3 3] ['l', 'o', 'o', 'o']
[2 3 3 3] ['l', 'o', 'o', 'o']
[2 3 2 3] ['l', 'o', 'l', 'o']
[2 3 2 3] ['l', 'o', 'l', 'o']
[2 3 2 3] ['l', 'o', 'l', 'o']
[2 3 2 3] ['l', 'o', 'l', 'o']
[2 3 2 3] ['l', 'o', 'l', 'o']
[2 3 2 3] ['l', 'o', 'l', 'o']
[2 3 2 3] ['l', 'o', 'l', 'o']
[2 3 2 3] ['l', 'o', 'l', 'o']
[2 3 2 3] ['l', 'o', 'l', 'o']
[2 3 2 3] ['l', 'o', 'l', 'o']
[2 3 2 3] ['l', 'o', 'l', 'o']
[2 3 2 3] ['l', 'o', 'l', 'o']
[2 3 2 3] ['l', 'o', 'l', 'o']
[2 3 2 3] ['l', 'o', 'l', '